# Preppin Data 2023 Week 07

Challenge: https://preppindata.blogspot.com/2023/02/2023-week-7-flagging-fraudulent.html

#### Load data

In [2]:
import pandas as pd

In [320]:
acc_hol = pd.read_csv('Account Holders.csv')
acc_info = pd.read_csv('Account Information.csv')
txn_det = pd.read_csv('Transaction Detail.csv')
txn_pth = pd.read_csv('Transaction Path.csv')

#### Transaction Path

In [321]:
txn_pth = txn_pth.rename(columns={'Account_From':'Account From', 'Account_To':'Account To'})

#### Account Information

In [322]:
# remove nulls
acc_info = acc_info.dropna()

##### split out joint accounts

In [323]:
# separate joint accounts
acc_j = acc_info[acc_info['Account Type'] == 'Joint']

In [324]:
# split account numbers to separate columns
acc_j[['ID1','ID2']] = acc_j['Account Holder ID'].str.split(',', expand=True)

C:\Users\amar\AppData\Local\Temp\ipykernel_3988\270181409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_j[['ID1','ID2']] = acc_j['Account Holder ID'].str.split(',', expand=True)
C:\Users\amar\AppData\Local\Temp\ipykernel_3988\270181409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_j[['ID1','ID2']] = acc_j['Account Holder ID'].str.split(',', expand=True)


In [325]:
acc_j

,Account Number,Account Type,Account Holder ID,Balance Date,Balance,ID1,ID2
4,10034341,Joint,"89920386, 97325900",2023-01-31,390.39,89920386,97325900
51,11360263,Joint,"62645453, 45852098",2023-01-31,136.79,62645453,45852098
115,13426667,Joint,"25339222, 48002566",2023-01-31,70.47,25339222,48002566
145,14448009,Joint,"86296209, 38678655",2023-01-31,958.46,86296209,38678655
160,14907946,Joint,"45905894, 80406841",2023-01-31,845.72,45905894,80406841
...,...,...,...,...,...,...,...
2866,95446510,Joint,"19187973, 09130396",2023-01-31,581.43,19187973,09130396
2914,97265555,Joint,"35624995, 68766400",2023-01-31,73.24,35624995,68766400
2934,97762952,Joint,"76833529, 34547171",2023-01-31,411.87,76833529,34547171
2947,97988499,Joint,"57370145, 00711968",2023-01-31,344.28,57370145,00711968


In [326]:
#separate into separate tables then merge
acc_j1 = acc_j[['Account Number','Account Type','Balance Date','Balance','ID1']]

In [327]:
acc_j1 = acc_j1.rename(columns={'ID1':'Account Holder ID'})

In [328]:
acc_j2 = acc_j[['Account Number','Account Type','Balance Date','Balance','ID2']]

In [329]:
acc_j2 = acc_j2.rename(columns={'ID2':'Account Holder ID'})

In [330]:
acc_j3 = pd.concat([acc_j1, acc_j2])

In [331]:
acc_j3

,Account Number,Account Type,Balance Date,Balance,Account Holder ID
4,10034341,Joint,2023-01-31,390.39,89920386
51,11360263,Joint,2023-01-31,136.79,62645453
115,13426667,Joint,2023-01-31,70.47,25339222
145,14448009,Joint,2023-01-31,958.46,86296209
160,14907946,Joint,2023-01-31,845.72,45905894
...,...,...,...,...,...
2866,95446510,Joint,2023-01-31,581.43,09130396
2914,97265555,Joint,2023-01-31,73.24,68766400
2934,97762952,Joint,2023-01-31,411.87,34547171
2947,97988499,Joint,2023-01-31,344.28,00711968


In [332]:
# remove joint from original table and add back in
j_index = acc_info[acc_info['Account Type'] == 'Joint'].index
acc_info.drop(j_index, inplace=True)

In [333]:
acc_info = pd.concat([acc_info, acc_j3])

In [334]:
acc_info

,Account Number,Account Type,Account Holder ID,Balance Date,Balance
0,10005367,Platinum,70390615,2023-01-31,728.25
1,10011977,Basic,20123998,2023-01-31,676.54
2,10024680,Platinum,54374080,2023-01-31,567.46
3,10031238,Basic,97027297,2023-01-31,576.52
5,10079965,Basic,89460366,2023-01-31,799.27
...,...,...,...,...,...
2866,95446510,Joint,09130396,2023-01-31,581.43
2914,97265555,Joint,68766400,2023-01-31,73.24
2934,97762952,Joint,34547171,2023-01-31,411.87
2947,97988499,Joint,00711968,2023-01-31,344.28


#### Account Holders

In [335]:
acc_hol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Account Holder ID      3072 non-null   int64 
 1   Name                   3072 non-null   object
 2   Date of Birth          3072 non-null   object
 3   Contact Number         3072 non-null   int64 
 4   First Line of Address  3072 non-null   object
dtypes: int64(2), object(3)
memory usage: 120.1+ KB


In [336]:
acc_hol['Contact Number'] = '0' + acc_hol['Contact Number'].astype('str')

In [337]:
acc_hol

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address
0,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park
1,20123998,Maynord Surgeoner,21/08/1997,07716107305,6422 Buena Vista Plaza
2,54374080,Giraldo Kimbley,22/03/1995,07489940612,93005 Summer Ridge Avenue
3,97027297,Blake Dudson,30/06/1955,07253587445,2 Huxley Hill
4,89920386,Ajay Douce,19/12/1930,07395580534,90176 Miller Alley
...,...,...,...,...,...
3067,66659633,Mata Brownett,16/07/1931,07314541365,5 Kenwood Park
3068,94872412,Tabby Matteotti,13/10/1962,07586210387,7517 Aberg Plaza
3069,90069774,Cyndia Fosse,09/03/1961,07494132554,42 Trailsway Point
3070,45810412,Arch Segrave,05/06/2000,07289510957,14 3rd Center


#### Merge

In [338]:
acc_hol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Account Holder ID      3072 non-null   int64 
 1   Name                   3072 non-null   object
 2   Date of Birth          3072 non-null   object
 3   Contact Number         3072 non-null   object
 4   First Line of Address  3072 non-null   object
dtypes: int64(1), object(4)
memory usage: 120.1+ KB


In [339]:
acc_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3072 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Account Number     3072 non-null   int64  
 1   Account Type       3072 non-null   object 
 2   Account Holder ID  3072 non-null   object 
 3   Balance Date       3072 non-null   object 
 4   Balance            3072 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 144.0+ KB


In [340]:
txn_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8781 entries, 0 to 8780
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8781 non-null   int64  
 1   Transaction Date  8781 non-null   object 
 2   Value             8781 non-null   float64
 3   Cancelled?        8781 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 274.5+ KB


In [341]:
txn_pth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8781 entries, 0 to 8780
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Transaction ID  8781 non-null   int64
 1   Account To      8781 non-null   int64
 2   Account From    8781 non-null   int64
dtypes: int64(3)
memory usage: 205.9 KB


In [343]:
acc_info['Account Holder ID'] = acc_info['Account Holder ID'].astype('int')

In [359]:
# accounts table 
df = acc_hol.merge(acc_info, how='left', left_on='Account Holder ID', right_on='Account Holder ID')

In [345]:
df

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address,Account Number,Account Type,Balance Date,Balance
0,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25
1,20123998,Maynord Surgeoner,21/08/1997,07716107305,6422 Buena Vista Plaza,10011977,Basic,2023-01-31,676.54
2,54374080,Giraldo Kimbley,22/03/1995,07489940612,93005 Summer Ridge Avenue,10024680,Platinum,2023-01-31,567.46
3,97027297,Blake Dudson,30/06/1955,07253587445,2 Huxley Hill,10031238,Basic,2023-01-31,576.52
4,89920386,Ajay Douce,19/12/1930,07395580534,90176 Miller Alley,10034341,Joint,2023-01-31,390.39
...,...,...,...,...,...,...,...,...,...
3067,66659633,Mata Brownett,16/07/1931,07314541365,5 Kenwood Park,99760030,Gold,2023-01-31,988.00
3068,94872412,Tabby Matteotti,13/10/1962,07586210387,7517 Aberg Plaza,99791709,Gold,2023-01-31,15.05
3069,90069774,Cyndia Fosse,09/03/1961,07494132554,42 Trailsway Point,99877007,Basic,2023-01-31,582.00
3070,45810412,Arch Segrave,05/06/2000,07289510957,14 3rd Center,99937043,Joint,2023-01-31,918.42


In [346]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3072 entries, 0 to 3071
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Account Holder ID      3072 non-null   int64  
 1   Name                   3072 non-null   object 
 2   Date of Birth          3072 non-null   object 
 3   Contact Number         3072 non-null   object 
 4   First Line of Address  3072 non-null   object 
 5   Account Number         3072 non-null   int64  
 6   Account Type           3072 non-null   object 
 7   Balance Date           3072 non-null   object 
 8   Balance                3072 non-null   float64
dtypes: float64(1), int64(2), object(6)
memory usage: 240.0+ KB


In [360]:
# transactions table
df1 = txn_det.merge(txn_pth, how='left', left_on='Transaction ID', right_on='Transaction ID')

In [348]:
df1

,Transaction ID,Transaction Date,Value,Cancelled?,Account To,Account From
0,1957155,2023-02-01,128.78,N,27356852,76206810
1,28234510,2023-02-01,163.82,N,28745450,87373821
2,33688648,2023-02-01,54.71,N,48271608,98821521
3,41670299,2023-02-01,88.10,N,34128127,80808205
4,42825784,2023-02-01,217.22,Y,25006771,81049128
...,...,...,...,...,...,...
8776,9881408962,2023-02-14,38.18,N,19920990,25099734
8777,9889326485,2023-02-14,126.71,N,86864320,22404690
8778,9892097130,2023-02-14,157.43,N,44912164,88933315
8779,9951297137,2023-02-14,120.16,N,54288062,97521278


In [371]:
# combined table
df2 = df.merge(df1, how='left', left_on='Account Number', right_on='Account From')

In [372]:
df2

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address,Account Number,Account Type,Balance Date,Balance,Transaction ID,Transaction Date,Value,Cancelled?,Account To,Account From
0,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25,1.753187e+09,2023-02-02,1097.60,N,65199307.0,10005367.0
1,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25,8.072095e+09,2023-02-04,67.01,N,34681193.0,10005367.0
2,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25,3.011216e+09,2023-02-05,108.26,N,44427272.0,10005367.0
3,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25,1.711865e+09,2023-02-08,65.49,N,43772330.0,10005367.0
4,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25,9.755066e+09,2023-02-10,27.61,N,39620163.0,10005367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9147,83107272,Niko Rudderham,11/12/2003,07312463759,14 3rd Center,99937043,Joint,2023-01-31,918.42,6.315649e+09,2023-02-02,142.45,N,78362587.0,99937043.0
9148,83107272,Niko Rudderham,11/12/2003,07312463759,14 3rd Center,99937043,Joint,2023-01-31,918.42,8.195984e+09,2023-02-03,52.49,N,85826354.0,99937043.0
9149,83107272,Niko Rudderham,11/12/2003,07312463759,14 3rd Center,99937043,Joint,2023-01-31,918.42,2.016923e+09,2023-02-06,92.42,N,42783101.0,99937043.0
9150,83107272,Niko Rudderham,11/12/2003,07312463759,14 3rd Center,99937043,Joint,2023-01-31,918.42,7.354862e+09,2023-02-09,122.91,N,35869347.0,99937043.0


#### Filter out transactions

In [373]:
# check for nulls to remove later
df2['Account From'].isnull().sum()

177

In [377]:
# filter out cancelled
index_can = df2[df2['Cancelled?'] == 'Y'].index
df2.drop(index_can, inplace=True)

In [376]:
# remove transactions less than £1,000
index_val = df2[df2['Value'] <= 1000].index
df2.drop(index_val, inplace=True)

In [353]:
df2

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address,Account Number,Account Type,Balance Date,Balance,Transaction ID,Transaction Date,Value,Cancelled?,Account To,Account From
0,70390615,Mahmoud Hehnke,28/08/1995,07479286250,18535 Loftsgordon Park,10005367,Platinum,2023-01-31,728.25,1.753187e+09,2023-02-02,1097.6,N,65199307.0,10005367.0
33,41402857,Federico Cathro,13/12/1985,07283906757,12 Green Hill,10231977,Basic,2023-01-31,61.63,4.230173e+09,2023-02-09,1410.1,N,68315462.0,10231977.0
35,8882143,Kizzee Puddephatt,18/10/2003,07598835878,72404 Maywood Avenue,10232603,Gold,2023-01-31,732.55,7.252110e+09,2023-02-02,1190.9,N,69344396.0,10232603.0
38,8882143,Kizzee Puddephatt,18/10/2003,07598835878,72404 Maywood Avenue,10232603,Gold,2023-01-31,732.55,9.161410e+09,2023-02-10,1128.4,N,91972687.0,10232603.0
41,84620072,Kalvin Duetschens,27/03/1965,07727619510,19032 Duke Parkway,10243581,Gold,2023-01-31,104.27,7.934677e+09,2023-02-12,1482.9,N,69664627.0,10243581.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,15559813,Kerrill Dunstan,18/02/1952,07605122700,921 Merrick Way,99729369,Basic,2023-01-31,751.01,5.567412e+09,2023-02-03,1619.6,N,71383376.0,99729369.0
9131,49193930,Hilary Crole,01/11/1974,07429810282,97581 Nevada Alley,99730589,Gold,2023-01-31,988.44,NaN,NaN,NaN,NaN,NaN,NaN
9132,56623581,Jana Sibthorp,02/03/1938,07302718672,48 Sutteridge Plaza,99734848,Gold,2023-01-31,354.29,4.286865e+09,2023-02-12,1342.4,N,42726921.0,99734848.0
9134,66659633,Mata Brownett,16/07/1931,07314541365,5 Kenwood Park,99760030,Gold,2023-01-31,988.00,2.022636e+08,2023-02-02,1028.8,N,22035411.0,99760030.0


In [374]:
# remove platinum accounts
index_plat = df2[df2['Account Type'] == 'Platinum'].index
df2.drop(index_plat, inplace=True)

In [378]:
df2

,Account Holder ID,Name,Date of Birth,Contact Number,First Line of Address,Account Number,Account Type,Balance Date,Balance,Transaction ID,Transaction Date,Value,Cancelled?,Account To,Account From
33,41402857,Federico Cathro,13/12/1985,07283906757,12 Green Hill,10231977,Basic,2023-01-31,61.63,4.230173e+09,2023-02-09,1410.1,N,68315462.0,10231977.0
35,8882143,Kizzee Puddephatt,18/10/2003,07598835878,72404 Maywood Avenue,10232603,Gold,2023-01-31,732.55,7.252110e+09,2023-02-02,1190.9,N,69344396.0,10232603.0
38,8882143,Kizzee Puddephatt,18/10/2003,07598835878,72404 Maywood Avenue,10232603,Gold,2023-01-31,732.55,9.161410e+09,2023-02-10,1128.4,N,91972687.0,10232603.0
41,84620072,Kalvin Duetschens,27/03/1965,07727619510,19032 Duke Parkway,10243581,Gold,2023-01-31,104.27,7.934677e+09,2023-02-12,1482.9,N,69664627.0,10243581.0
58,95870139,Ellissa Blanchette,30/07/2000,07252539589,56762 Green Ridge Circle,10449180,Basic,2023-01-31,228.24,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,15559813,Kerrill Dunstan,18/02/1952,07605122700,921 Merrick Way,99729369,Basic,2023-01-31,751.01,5.567412e+09,2023-02-03,1619.6,N,71383376.0,99729369.0
9131,49193930,Hilary Crole,01/11/1974,07429810282,97581 Nevada Alley,99730589,Gold,2023-01-31,988.44,NaN,NaN,NaN,NaN,NaN,NaN
9132,56623581,Jana Sibthorp,02/03/1938,07302718672,48 Sutteridge Plaza,99734848,Gold,2023-01-31,354.29,4.286865e+09,2023-02-12,1342.4,N,42726921.0,99734848.0
9134,66659633,Mata Brownett,16/07/1931,07314541365,5 Kenwood Park,99760030,Gold,2023-01-31,988.00,2.022636e+08,2023-02-02,1028.8,N,22035411.0,99760030.0


#### Drop NA and export

In [381]:
df3 = df2.dropna()

In [382]:
df3 = df3[['Transaction ID','Account To','Transaction Date','Value','Account Number','Account Type',\
     'Balance Date','Balance','Name','Date of Birth','Contact Number','First Line of Address']]

In [383]:
df3.to_csv('2023W07_output.csv',index=False)